## python爬虫示例
[作者：M小白](mwhitelab.com)

公众号：技术杂学铺

In [1]:
# 导入必要函数包
import requests                   # 向服务器发送并获取数据
import json                       # 分析json数据格式
from bs4 import BeautifulSoup     # 分析html数据
import sqlite3                    # 数据库

In [2]:
url = "http://www.baidu.com" # 给定网址
response = requests.get(url) # 使用请求并获取数据

In [3]:
'''
响应状态码
200：成功
301：跳转
403：无权限访问
404：文件不存在
502：服务器错误
'''
response

<Response [200]>

In [4]:
# 编码为utf-8
response.encoding = 'utf-8'

In [5]:
# 获取数据
response.text

'<!DOCTYPE html>\r\n<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charset=utf-8><meta http-equiv=X-UA-Compatible content=IE=Edge><meta content=always name=referrer><link rel=stylesheet type=text/css href=http://s1.bdstatic.com/r/www/cache/bdorz/baidu.min.css><title>百度一下，你就知道</title></head> <body link=#0000cc> <div id=wrapper> <div id=head> <div class=head_wrapper> <div class=s_form> <div class=s_form_wrapper> <div id=lg> <img hidefocus=true src=//www.baidu.com/img/bd_logo1.png width=270 height=129> </div> <form id=form name=f action=//www.baidu.com/s class=fm> <input type=hidden name=bdorz_come value=1> <input type=hidden name=ie value=utf-8> <input type=hidden name=f value=8> <input type=hidden name=rsv_bp value=1> <input type=hidden name=rsv_idx value=1> <input type=hidden name=tn value=baidu><span class="bg s_ipt_wr"><input id=kw name=wd class=s_ipt value maxlength=255 autocomplete=off autofocus></span><span class="bg s_btn_wr"><input type=submit id=su

In [6]:
# 使用BeautifulSoup对html文件进行分析
soup = BeautifulSoup(response.content, "lxml")

In [7]:
# 找到所有链接 find_all中第一个参数为html的标签
soup.find_all('a')

[<a class="mnav" href="http://news.baidu.com" name="tj_trnews">新闻</a>,
 <a class="mnav" href="http://www.hao123.com" name="tj_trhao123">hao123</a>,
 <a class="mnav" href="http://map.baidu.com" name="tj_trmap">地图</a>,
 <a class="mnav" href="http://v.baidu.com" name="tj_trvideo">视频</a>,
 <a class="mnav" href="http://tieba.baidu.com" name="tj_trtieba">贴吧</a>,
 <a class="lb" href="http://www.baidu.com/bdorz/login.gif?login&amp;tpl=mn&amp;u=http%3A%2F%2Fwww.baidu.com%2f%3fbdorz_come%3d1" name="tj_login">登录</a>,
 <a class="bri" href="//www.baidu.com/more/" name="tj_briicon" style="display: block;">更多产品</a>,
 <a href="http://home.baidu.com">关于百度</a>,
 <a href="http://ir.baidu.com">About Baidu</a>,
 <a href="http://www.baidu.com/duty/">使用百度前必读</a>,
 <a class="cp-feedback" href="http://jianyi.baidu.com/">意见反馈</a>]

In [8]:
# 找到所有类为'mnav'的链接
soup.find_all('a',class_='mnav')

[<a class="mnav" href="http://news.baidu.com" name="tj_trnews">新闻</a>,
 <a class="mnav" href="http://www.hao123.com" name="tj_trhao123">hao123</a>,
 <a class="mnav" href="http://map.baidu.com" name="tj_trmap">地图</a>,
 <a class="mnav" href="http://v.baidu.com" name="tj_trvideo">视频</a>,
 <a class="mnav" href="http://tieba.baidu.com" name="tj_trtieba">贴吧</a>]

## 爬取流浪地球豆瓣评价

In [9]:
import pandas as pd # 便于之后数据处理

In [10]:
star_list = []     # 评分列表
comment_list = []  # 评论列表

In [11]:
for k in range(10):
    url = "https://movie.douban.com/subject/26266893/comments?start="+str(k*20)+"&limit=20&sort=new_score&status=P&comments_only=1" # 给定网址
    response = requests.get(url) # 使用请求并获取数据
    response.encoding = 'utf-8'
    params = json.loads(response.text) # 处理html数据
    html = params['html']
    soup = BeautifulSoup(html, "lxml")
    comment = soup.find_all('div',class_='comment')
    for i in range(len(comment)):
        comment_check = comment[i].find_all('span',class_='rating')
        if(len(comment_check)):
            star = str(comment_check[0])[20]
            short = comment[i].find_all('span',class_='short')[0].text
            star_list.append(star)
            comment_list.append(short)
    print(k)

0
1
2
3
4
5
6
7
8
9


In [12]:
comments = pd.DataFrame({'star':star_list,'comment':comment_list})

In [13]:
comments

,comment,star
0,电影比预期要更恢弘磅礴，晨昏线过后的永夜、火种计划、让地球流浪、木星推动地球…等等大小设定，...,4
1,一个悲伤的故事：太阳都要毁灭，地球都要流浪了，我国的校服还是这么丑......,4
2,1.终于，轮到我们仰望星空。2.后启示录死亡废墟，赛博朋克地下城，以及烟波浩渺的末日想象，缔...,5
3,真为吴京的演技尴尬，总是摆出一副大义凌然的样子，好奇为什么刘的作品中总有这种傻逼般的圣母存在...,1
4,还能更土更儿戏一点吗？毫无思考仅靠煽动，毫无敬畏仅余妄想。好的科幻片应该首先承认人类的无知，...,1
5,三星鼓励一下吧，四个字：太儿戏了。硬科幻和硬要科幻是两回事。,3
6,快要把我震死在影院座椅上了。影片有科幻设定而无科幻精神，每个人物都处于被集体主义长期坑害的大...,1
7,失望 一群聒噪的角色让人忍不住觉得地球还是毁灭了比较好,2
8,野心远远大于能力的作品。大刘小说打底，导演用影视化视觉实现了大部分想象，敢想敢做！远超预期，...,1
9,求求编剧，人类都快失去地球了，生存才是最重要的。请别再bb这点家庭琐事和煽情烂梗。白瞎了刘慈...,1
